In [7]:
%reload_ext autoreload
%autoreload 2

import voxelmorph as vxm
import matplotlib.pyplot as plt
import neurite as ne
import os
import numpy as np
import subtle.subtle_preprocess as supre
import subtle.utils.io as suio
import subtle.subtle_metrics as su_metrics
import pydicom
from glob import glob

import keras.layers as KL
from keras.models import Model

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (10, 8)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<Figure size 720x576 with 0 Axes>

In [9]:
n_cascades = 3
img_shape = (128, 256, 256)
name = 'cascade'

source = KL.Input(shape=(*img_shape, 1), name='{}_source_input'.format(name))
target = KL.Input(shape=(*img_shape, 1), name='{}_target_input'.format(name))

nets = [vxm.networks.AIRNet(inshape=(128, 256, 256), verbose=0).model for _ in np.arange(n_cascades)]

src_moved = source

for c_idx in np.arange(n_cascades):
    src_moved = nets[c_idx]([src_moved, target])
    
model = Model(inputs=[source, target], outputs=[src_moved])
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cascade_source_input (InputLaye (None, 128, 256, 256 0                                            
__________________________________________________________________________________________________
cascade_target_input (InputLaye (None, 128, 256, 256 0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 128, 256, 256 13016380    cascade_source_input[0][0]       
                                                                 cascade_target_input[0][0]       
__________________________________________________________________________________________________
model_6 (Model)                 (None, 128, 256, 256 13016380    model_5[1][0]              

In [14]:
s = str(vxm.networks.AIRNet)
print(s.replace('\'', '').replace('<', '').replace('>', '').split('.')[-1].lower())

airnet
